In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("../input/us-accidents/US_Accidents_Dec20.csv")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum().sort_values(ascending=False)

In [ ]:
missing_percent = df.isna().sum().sort_values(ascending=False) / len(df)
missing_percent

In [ ]:
type(missing_percent)

In [ ]:
missing_percent[missing_percent != 0].plot(kind='bar')

## Exploratory Analysis and Visualization

**Column to Analyze**
1. City
2. Start Time
3. Start Lat, Start Lag
4. Temparature
5. Weather Condition

In [ ]:
cities = df["City"].unique()
len(cities)

In [ ]:
cities_by_accident = df["City"].value_counts().sort_values(ascending=False)
cities_by_accident.head(15).plot(kind='bar');

In [ ]:
"New York" in df["City"]

In [ ]:
high_accident_cities = cities_by_accident[cities_by_accident >= 1000]
low_accident_cities = cities_by_accident[cities_by_accident < 1000]

In [ ]:
len(high_accident_cities)

In [ ]:
len(low_accident_cities)

In [ ]:
sns.histplot(high_accident_cities, log_scale=True);

In [ ]:
sns.histplot(low_accident_cities, log_scale=True);

In [ ]:
len([cities_by_accident == 1]) 

In [ ]:
df["Start_Time"] = pd.to_datetime(df["Start_Time"])

In [ ]:
df["Start_Time"][0]

In [ ]:
sns.distplot(df["Start_Time"].dt.hour, bins=24, kde=False, norm_hist=True)

In [ ]:
sns.distplot(df["Start_Time"].dt.dayofweek, bins=7, kde=False, norm_hist=True)

Is the distribution of accidents by hour the same on weekends as on weekdays.

In [ ]:
sundays_start_time = df.Start_Time[df["Start_Time"].dt.dayofweek == 6]
sns.distplot(sundays_start_time.dt.hour, bins=24, kde=False, norm_hist=True)

In [ ]:
monday_start_time = df.Start_Time[df["Start_Time"].dt.dayofweek == 0]
sns.distplot(monday_start_time.dt.hour, bins=24, kde=False, norm_hist=True)

On Sundays, the peak occurs between 10 am and 3 pm, unlike weekdays

In [ ]:
df_2019 = df[df["Start_Time"].dt.year == 2019]
df_2019_Bing = df_2019[df_2019.Source == 'MapQuest']
sns.distplot(df_2019_Bing["Start_Time"].dt.month, bins=12, kde=False, norm_hist=True)

Can you explain the month-wise trend of accidents?
* Much data is missing for 2016. Maybe even 2017.
* There seems to be some issue with the Bing data

In [ ]:
df["Source"].value_counts().plot(kind='pie')

**Consider excluding Bing data, seems to have issues.**

## Start Latitude and Longitude

In [ ]:
df["Start_Lat"].head()

In [ ]:
df["Start_Lng"].head()

In [ ]:
sample_df = df.sample(int(0.1 * len(df)))

In [ ]:
sns.scatterplot(x=sample_df["Start_Lng"], y=sample_df["Start_Lat"], size=0.001)

In [ ]:
import folium

In [ ]:
zip(list(df["Start_Lat"]), list(df["Start_Lng"]))

In [ ]:
from folium.plugins import HeatMap

In [ ]:
sample_df = df.sample(int(0.001 * len(df)))

In [ ]:
lat_lon_pairs = list(zip(list(sample_df["Start_Lat"]), list(sample_df["Start_Lng"])))

In [ ]:
map = folium.Map()
HeatMap(lat_lon_pairs).add_to(map)
map

## Summary and Conclusion

1. No data from New York
2. Less than 5% of cities have more than 1000 yearly accidents
3. over 1200 cities have reported just one accident(need to investigate).